In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
from keras import regularizers
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from tensorflow.keras import backend as K
np.random.seed(10)
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from absl import logging

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5"
embed = hub.KerasLayer(module_url)

In [ ]:
train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
print('There are {} rows and {} columns in train'.format(train.shape[0],train.shape[1]))
print('There are {} rows and {} columns in train'.format(test.shape[0],test.shape[1]))

In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,5))
tweet_len=train[train['target']==1]['text'].str.split().map(lambda x: len(x))
ax1.hist(tweet_len,color='red')
ax1.set_title('disaster tweets')
tweet_len=train[train['target']==0]['text'].str.split().map(lambda x: len(x))
ax2.hist(tweet_len,color='green')
ax2.set_title('Not disaster tweets')
fig.suptitle('Words in a tweet')
plt.show()

In [ ]:
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
from collections import Counter
train.keyword = train['keyword'].str.replace("[^a-zA-Z#]", " ")
keyword = train.keyword[train.keyword.notnull()].tolist()
keyword = Counter(keyword)
keywords = pd.DataFrame(keyword.most_common(10), columns=['Keyword', 'Count'])
sns.set(rc={'figure.figsize':(14,6)})
sns.barplot(data = keywords, x = 'Keyword', y = 'Count')
plt.title("Most Common Keywords")
plt.show()

In [ ]:
sns.countplot(train['target'])
plt.title("Distribution Of Target")
sns.set(rc={'figure.figsize':(10,8)})
plt.show()

In [ ]:
'''
import re
import string
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

train['text'] = train.text.apply(lambda x: clean_text(x))
train['text'] = train['text'].str.replace("[^a-zA-Z#]", " ")
train['text'] = train['text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
'''

In [ ]:
import string
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

train['text']=train['text'].apply(lambda x : remove_URL(x))
test['text']=test['text'].apply(lambda x : remove_URL(x))

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)


train['text']=train['text'].apply(lambda x : remove_html(x))
test['text']=test['text'].apply(lambda x : remove_html(x))


def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

remove_emoji("Omg another Earthquake 😔😔")


train['text']=train['text'].apply(lambda x : remove_emoji(x))
test['text']=test['text'].apply(lambda x : remove_emoji(x))

def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

example="I am a #king"
print(remove_punct(example))

train['text']=train['text'].apply(lambda x : remove_punct(x))
test['text']=test['text'].apply(lambda x : remove_punct(x))

!pip install pyspellchecker

from spellchecker import SpellChecker

'''
pell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)
        
text = "corect me plese"
correct_spellings(text)

train['text']=train['text'].apply(lambda x : correct_spellings(x))
test['text']=test['text'].apply(lambda x : correct_spellings(x))
'''



In [ ]:
train.keyword = train.keyword.fillna("")
train['new_text'] = train.text + " " +train.keyword

test.keyword = test.keyword.fillna("")
test['text'] = test.text + " " +test.keyword




In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train.new_text.values, train.target.values, test_size=0.2, random_state=42)
test_data = test.text.values

In [ ]:
def build_model(embed):
    model = Sequential([
        Input(shape=[], dtype=tf.string),
        embed,
        Dense(512,activation='relu'),
        BatchNormalization(),
        Dropout(0.1),
        Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.02)),
        BatchNormalization(),
        Dropout(0.05),
        Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.02)),
        BatchNormalization(),
        Dropout(0.05),
        Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.02)),
        BatchNormalization(),
        Dense(1, activation='sigmoid')
    ])
    model.compile(Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    '''
        Dense(1024, activation='relu'),
        BatchNormalization(),
        Dropout(0.1),
    '''
    return model

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
#module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5"
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

embed_new = hub.Module(module_url)
with tf.compat.v1.Session() as session:
  session.run([tf.compat.v1.global_variables_initializer(), tf.compat.v1.tables_initializer()])
  sentences_embeddings = session.run(embed_new(X_train))
  sentences_embeddings_test = session.run(embed_new(X_test))
  test_embedding = session.run(embed_new(test_data))

train_embeddings = np.array(sentences_embeddings)
test_embeddings = np.array(sentences_embeddings_test)
test_data_embeddings = np.array(test_embedding)
  
    


In [ ]:
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(train_embeddings, y_train)
clf.score(test_embeddings,y_test)
sub = clf.predict(test_data_embeddings)
subm = pd.DataFrame()
subm['id'] = test['id']
subm['target'] = sub.round().astype(int)
subm.to_csv("pred_svm.csv", index = False)

In [ ]:
from keras.layers import LSTM
model = build_model(embed)
model.summary()

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('model.h5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

In [ ]:
print(X_train[0])
print(y_train.shape)
with tf.compat.v1.Session() as session:
    tf.compat.v1.keras.backend.set_session(session)
    session.run([tf.compat.v1.global_variables_initializer(), tf.compat.v1.tables_initializer()])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test,y_test),
        epochs=35,
        callbacks=[earlyStopping,mcp_save,reduce_lr_loss],
        batch_size=16
    )

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(2,2,1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
#plt.show()
plt.subplot(2,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
with tf.Session() as session:
    tf.compat.v1.keras.backend.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('model_new.h5')
    y_pred = model.predict(X_test)
    
from sklearn.metrics import confusion_matrix, classification_report
#print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred.round().astype(int)))

In [ ]:
with tf.Session() as session:
    tf.compat.v1.keras.backend.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('model_new.h5')
    sub = model.predict(test_data)
    

subm = pd.DataFrame()
subm['id'] = test['id']
subm['target'] = sub.round().astype(int)
subm.to_csv("pred_new.csv", index = False)